In [38]:
import sqlalchemy as db

In [39]:
# sqlite://<nohostname>/<path>
# where <path> is relative:
engine = db.create_engine("sqlite:///restaurant.db")

In [40]:
connection = engine.connect()
metadata = db.MetaData()


In [41]:


ingredient_table = db.Table('Ingredient', metadata,
              db.Column('nom_ingredient', db.String(), primary_key=True),
              db.Column('prix_achat', db.Float(), default=0.0),
              )



In [42]:


pays_table = db.Table('Pays', metadata,
              db.Column('pays', db.String(), primary_key=True),
              )



In [43]:


restaurant_table = db.Table('Restaurant', metadata,
              db.Column('code_postal', db.String(), primary_key=True),
              db.Column('parking', db.Integer()),
              db.Column('espace_enfant', db.Integer()),
              db.Column('pmr', db.Integer()),
              db.Column('borne', db.Integer()),
              db.Column('capacite', db.Integer()),
              db.Column('pays', db.String(), db.ForeignKey("Pays.pays")),
              )



In [44]:


stock_table = db.Table('Stock', metadata,
              db.Column('id_stock', db.Integer(),primary_key=True),
              db.Column('nom_ingredient', db.String(),db.ForeignKey("Ingredient.nom_ingredient")),
              db.Column('code_postal', db.String(),db.ForeignKey("Restaurant.code_postal")),
              db.Column('quantite', db.Integer()),
              )



In [45]:


item_table = db.Table('Item', metadata,
              db.Column('nom_item', db.String(),primary_key=True),
              db.Column('type', db.String()),
              db.Column('prix', db.Float()),
              )



In [46]:

carte_item_table = db.Table('CarteItem', metadata,
              db.Column('id_carte_item', db.Integer(),primary_key=True),
              db.Column('nom_item', db.String(),db.ForeignKey("Item.nom_item")),
              db.Column('pays', db.String(),db.ForeignKey("Pays.pays")),
              )

In [47]:
metadata.create_all(engine)